In [10]:
import datacube
from dc_notebook_utilities import create_platform_product_gui, display_map
from utils.data_cube_utilities.data_access_api import DataAccessApi

In [13]:
dc = datacube.Datacube(config="/home/localuser/.datacube.conf")
api = DataAccessApi(config="/home/localuser/.datacube.conf")
from utils.data_cube_utilities.data_access_api import DataAccessApi

In [14]:

#Get all the current datacube products
products = dc.list_products()

In [15]:
###If we decide we are avoiding widgets run this block and skip the next 2 cells
platform = "LANDSAT_7"
product = "ls7_ledaps_vietnam"

In [16]:
# ###in case we go with widgets:
# #get platforms
# platform_names = list(set(products.platform))
# #get products
# product_names = list(products.name)

# #List viable options
# product_values = create_platform_product_gui(platform_names, product_names, dc)

In [17]:
# platform, product  = product_values
# platform = platform[0]
# product = product[0]

In [18]:
lat, lon = products.resolution[products.platform == platform].any()

In [19]:
# Get the extents of the cube
descriptor = api.get_query_metadata(platform=platform, product=product)
# descriptor = api.get_descriptor(dict(platform = platform))[product[0]]

In [21]:
min_date, max_date = descriptor['time_extents']
min_lat, max_lat = descriptor['lat_extents']
min_lon, max_lon = descriptor['lon_extents']

min_date_str = str(min_date.year) + '-' + str(min_date.month) + '-' + str(min_date.day)

min_lat_rounded = round(min_lat, 3)
min_lon_rounded =  round(min_lon, 3)

max_date_str = str(max_date.year) + '-' + str(max_date.month) + '-' + str(max_date.day)

max_lat_rounded = round(max_lat, 3) #calculates latitude of the pixel's center
max_lon_rounded = round(max_lon, 3) #calculates longitude of the pixel's center

#display area
display_map(latitude = (min_lat_rounded, max_lat_rounded),longitude = (min_lon_rounded, max_lon_rounded))

NameError: name 'display_map' is not defined

In [ ]:
display_map(latitude = (min_lat_rounded, max_lat_rounded),longitude = (min_lon_rounded, max_lon_rounded))

In [ ]:
# Display metadata
generate_metadata_report(min_date_str, max_date_str, 
                         min_lon_rounded, max_lon_rounded, lon_dist,
                         min_lat_rounded, max_lat_rounded, lat_dist)

In [ ]:
extent_values = create_extents_gui(min_date_str, max_date_str,
                                   min_lon_rounded, max_lon_rounded,
                                   min_lat_rounded, max_lat_rounded)

In [ ]:
# Save form values
start_date = datetime.strptime(extent_values[0].value, '%Y-%m-%d')
end_date = datetime.strptime(extent_values[1].value, '%Y-%m-%d')
min_lon = extent_values[2].value
max_lon = extent_values[3].value
min_lat = extent_values[4].value
max_lat = extent_values[5].value

# Query the Data Cube
dataset_in = dc.load(platform=platform[0],
                     product=product[0],
                     time=(start_date, end_date),
                     lon=(min_lon, max_lon), 
                     lat=(min_lat, max_lat),
                     measurements = ["red", "green", "blue", 
                                     "nir", "swir1", "swir2", 
                                     "pixel_qa"])
if(len(dataset_in) is 0):
    print("dataset not loaded in, try another product")
else:
    print("successfully loaded data")